In [ ]:
# pip install requests
# pip install -U ipykernel
# pip install tabulate

In [1]:
import requests
URL = 'https://online-price-watch.consumer.org.hk/opw/opendata/pricewatch.json'
response = requests.get(URL)
result = response.json()
print('Count: %s' % (len(result)))

Count: 2101


In [13]:
import math

list = [
    item for item in result
    if len(item['prices']) > 1
]
print('Count: %s' % (len(list)))

for item in list:
    maxValue = 0
    minValue = math.inf
    highPrice = item['prices'][0]
    lowPrice = highPrice
    for price in item['prices']:
        value = float(price['price'])
        if value > maxValue:
            maxValue = value
            highPrice = price
        if value < minValue:
            minValue = value
            lowPrice = price
    
    item['high_price'] = highPrice
    item['low_price'] = lowPrice
    item['price_diff'] = maxValue - minValue
    item['price_diff_percent'] = math.floor(minValue / maxValue * 100)
    
print(list[0])

Count: 1594
{'code': 'P000000190', 'brand': {'en': 'Garden', 'zh-Hant': '嘉頓', 'zh-Hans': '嘉顿'}, 'name': {'en': 'Marbo Cake - Cherry & Raisin 75g', 'zh-Hant': '孖寶蛋糕 - 櫻桃提子 75克', 'zh-Hans': '孖宝蛋糕 - 樱桃提子 75克'}, 'cat1Name': {'en': 'Bakery / Cereals / Spreads', 'zh-Hant': '麵包蛋糕 / 穀類早餐 / 麵包醬', 'zh-Hans': '面包蛋糕 / 谷类早餐 / 面包酱'}, 'cat2Name': {'en': 'Cakes', 'zh-Hant': '蛋糕', 'zh-Hans': '蛋糕'}, 'cat3Name': {'en': 'Sponge cakes', 'zh-Hant': '海綿蛋糕', 'zh-Hans': '海绵蛋糕'}, 'prices': [{'supermarketCode': 'WELLCOME', 'price': '9.00'}, {'supermarketCode': 'JASONS', 'price': '9.50'}], 'offers': [{'supermarketCode': 'WELLCOME', 'en': '2 for $14.00', 'zh-Hant': '買2件$14.00', 'zh-Hans': '买2件$14.00'}, {'supermarketCode': 'JASONS', 'en': '2 for $15.00', 'zh-Hant': '買2件$15.00', 'zh-Hans': '买2件$15.00'}], 'price_diff': 0.5, 'price_diff_percent': 94, 'high_price': {'supermarketCode': 'JASONS', 'price': '9.50'}, 'low_price': {'supermarketCode': 'WELLCOME', 'price': '9.00'}}


In [6]:
from tabulate import tabulate

table = [
    ["Sun",696000,1989100000],
    ["Earth",6371,5973.6],
    ["Moon",1737,73.5],
    ["Mars",3390,641.85]
]
print(tabulate(table))

-----  ------  -------------
Sun    696000     1.9891e+09
Earth    6371  5973.6
Moon     1737    73.5
Mars     3390   641.85
-----  ------  -------------


In [48]:
def get_price_diff(item):
    return item.get('price_diff')

def get_price_diff_percent(item):
    return item.get('price_diff_percent')

list.sort(key=get_price_diff_percent)

table = [
    [
        item['code'], 
        # item['cat1Name']['zh-Hant'],
        '%s - %s' % (item['brand']['zh-Hant'], item['name']['zh-Hant']),
        item['price_diff'], 
        item['price_diff_percent'],
        item['low_price']['supermarketCode'], item['low_price']['price'], 
        item['high_price']['supermarketCode'], item['high_price']['price'], 
    ]
    for item in list
    if item['price_diff'] > 0
]

print(tabulate(
    table, 
    headers=[
        'Code', 'Name', 
        'Highest - Lowest', 
        'Lowest / Highest (%)',
        'Supermarket (Lowest Price)', 'Lowest Price',
        'Supermarket (Highest Price)', 'Highest Price',
    ],
    tablefmt="simple",
    floatfmt=".1f"
))

Code         Name                                                                                     Highest - Lowest    Lowest / Highest (%)  Supermarket (Lowest Price)      Lowest Price  Supermarket (Highest Price)      Highest Price
-----------  -------------------------------------------------------------------------------------  ------------------  ----------------------  ----------------------------  --------------  -----------------------------  ---------------
P000002213   特別好 - 熱帶雜果 439克                                                                              10.3                      28  PARKNSHOP                                4.2  AEON                                      14.5
P000000047   寿桃牌 (壽桃牌) - 幼滑上海麵 340克                                                                    6.9                      30  AEON                                     3.0  JASONS                                     9.9
P000004898   Bioderma - 深層卸妝潔膚水 500毫升                                   